In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import re
import bias_functions
import text_extraction
import asyncio
from scipy.stats import f_oneway

pip install 'aif360[inFairness]'
pip install 'aif360[OptimalTransport]'


In [2]:
#df = pd.read_csv('extracted_data.txt', names=['Name', 'Age', 'City', 'Gender', 'UG_College', 'PG_College', 'Degree', 'Current_Employer', 'Experience(in Yrs)'])

In [3]:
df = bias_functions.read_json_files('./uploads/parsed_json/')

In [4]:
df.head()

,gender,degree,institute,year,city,employer,experience,experience_range,keywords,age
0,NaN,Bachelor of Technology,UPTU,2002,Bareilly,Avida Technologies,0,GREATER THAN 10 YEAR,"[Sensitive, Functional Requirements Specificat...",0.0
1,Male,Bachelor of Engineering,College of Engineering and Technology,1993,Chennai,Mercantiler InfoTech pvt ltd,0,8-10 YEAR,"[Microsoft Visual Studio, User Experience Desi...",0.0
2,Male,Bachelor of Engineering,Bharathiyar University,1982,Bangalore,infinite computer solution,0,GREATER THAN 10 YEAR,"[Human Resources Management System, Hospital M...",1.0
3,NaN,Bachelor of Technology,Jawaharlal Nehru technological University,NaN,NaN,for Five,0,3-5 YEAR,"[Confident, Manhattan Warehouse Management Sys...",NaN
4,NaN,Master of Computer Applications,Telangana University,NaN,NaN,Cybage Software Pvt Ltd,0,GREATER THAN 10 YEAR,"[Hard Working, Independent, Mirroring, Potenti...",NaN


In [5]:
df.shape

(108, 10)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            33 non-null     object 
 1   degree            97 non-null     object 
 2   institute         92 non-null     object 
 3   year              70 non-null     object 
 4   city              72 non-null     object 
 5   employer          102 non-null    object 
 6   experience        108 non-null    object 
 7   experience_range  108 non-null    object 
 8   keywords          108 non-null    object 
 9   age               70 non-null     float64
dtypes: float64(1), object(9)
memory usage: 8.6+ KB


In [7]:
def process_column(column):
    def process_value(value):
        if isinstance(value, str):
            value = value.lower()
            value = value.strip()
            match = re.match(r'(\d+(\.\d+)?)\+', value)
            if match:
                return float(match.group(1))
            match = re.match(r'(\d+(\.\d+)?)\s*years?', value, re.IGNORECASE)
            if match:
                return float(match.group(1))
            if value.upper() == 'NULL':
                return np.nan
            try:
                return float(value)
            except ValueError:
                return np.nan  
        elif isinstance(value, (int, float)):
            return float(value)
        else:
            return np.nan  
    return column.apply(process_value)

In [8]:
df['experience'] = process_column(df['experience'])

In [9]:
def clean_column(column):
    def clean_value(value):
        if isinstance(value, str):
            value = value.lower()
            value = value.strip()
            value = value.replace('"', '')
            if any(char.isdigit() for char in value):
                return np.nan
            if value == 'NULL' or value == 'India' or value == 'Gender' or value == 'City':
                return np.nan
        return value
    return column.apply(clean_value)

In [10]:
df['gender'] = clean_column(df['gender'])

In [11]:
def clean_and_convert_column(column):
    def clean_value(value):
        if pd.isna(value): 
            return value
        value = str(value).strip() 
        try:
            return float(value)
        except ValueError:
            if any(char.isdigit() for char in value):
                try:
                    return float(value)
                except ValueError:
                    return np.nan
            else:
                return np.nan
    return column.apply(clean_value)

In [12]:
df['age'] = clean_and_convert_column(df['age'])

In [13]:
df['city'] = clean_column(df['city'])

In [14]:
df['employer'] = clean_column(df['employer'])
df['degree'] = clean_column(df['degree'])
df['institute'] = clean_column(df['institute'])

In [15]:
df.head()

,gender,degree,institute,year,city,employer,experience,experience_range,keywords,age
0,NaN,bachelor of technology,uptu,2002,bareilly,avida technologies,0.0,GREATER THAN 10 YEAR,"[Sensitive, Functional Requirements Specificat...",0.0
1,male,bachelor of engineering,college of engineering and technology,1993,chennai,mercantiler infotech pvt ltd,0.0,8-10 YEAR,"[Microsoft Visual Studio, User Experience Desi...",0.0
2,male,bachelor of engineering,bharathiyar university,1982,bangalore,infinite computer solution,0.0,GREATER THAN 10 YEAR,"[Human Resources Management System, Hospital M...",1.0
3,NaN,bachelor of technology,jawaharlal nehru technological university,NaN,NaN,for five,0.0,3-5 YEAR,"[Confident, Manhattan Warehouse Management Sys...",NaN
4,NaN,master of computer applications,telangana university,NaN,NaN,cybage software pvt ltd,0.0,GREATER THAN 10 YEAR,"[Hard Working, Independent, Mirroring, Potenti...",NaN


In [16]:
jd = """Landis+Gyr is a leading global provider of integrated energy management solutions. We measure and analyze energy utilization to generate empowering analytics for smart grid and infrastructure management, enabling utilities and consumers to reduce energy consumption. Our innovative and proven portfolio of software, services and intelligent sensor technology is a key driver to decarbonize the grid. With sales of USD 1.7 billion in FY 2022, Landis+Gyr employs over 7,000 talented people across 30 countries and five continents.

For more than a century, we've been pioneers in energy innovation, and we continue to lead the way toward a more sustainable and efficient energy landscape.

Join us at Landis+Gyr, where we manage energy better - since 1896.

Key Purpose of Role:

This role has responsiblity for development and expected to work in all areas of software engineering ( design, implement, review, document, test, support) as the demand dictates.

Experience Required: 3-10 years

Areas of Responsibility / Tasks:
• To design And Develop Applications software, APIs and enhancement in existing functionality.
• Involvement In Software Development Life-Cycle Activities (requirement gathering, elaboration, solicit clarification, WBS, Estimations, documentation).
• To develop specifications including Functional Requirements, Use Cases and Business Rules, Detailed Design, Code, And Test Procedures As Part Of Agile/Iterative Project Team.
• Document, Train And Support Operations And Software Quality Assurance TeamsPerform Code Reviews
• To perform unit/Integration testing
• Database Design and Maintenance.
• To Setup/ Configure/ Troubleshoot test environment
• Identify the quality KPI’s and ensure the compliance at each stage.
• Object-Oriented programming skills, Strong in .NET concepts
• Technology proficiency, C# with Windows Forms, WPF
• Database (Oracle/ SQL Server/My SQL/MS Access)
• Experience With Common Design Patterns,Documentation
• Experience using Defect management, Source Code Managenent, REQUIREMENT Management tools (Rational/TFS/Devops/Other...)
• Serial Communication/Device Communication (RS232/RS485/RF/GSM/GPRS)/DLMS

We are thrilled that you’ve considered Landis+Gyr to be a part of your professional journey.  As a part of the Landis+Gyr family, employees are offered a comprehensive and competitive package of benefits including medical, dental and vision coverage, life insurance, and a 401(k)-retirement plan featuring a generous company match incentive.  Additionally, we offer three (3) weeks of Paid Time Off (PTO) as well as eleven (11) paid holidays.  We also offer a tuition reimbursement program, optional pet insurance, supplemental medical coverage and a host of other perks to ensure the health, wellness, and enrichment of our team members. 

Pay is in the range of XY per year. The position is eligible for a 401(k) match and consideration for an annual bonus. The actual base salary offered depends on a variety of factors, which may include, as applicable, the qualifications of the individual applicant for the position, years of relevant experience, specific and unique skills, level of education attained, and certifications or other professional licenses held.  All candidates are encouraged to apply. 

All qualified applicants will receive consideration for employment without regard to race, color, religion, sex, sexual orientation, gender identity, national origin, disability, or status as a protected veteran. 

Submit your application – it takes less than 5 minutes! We will review and if selected to move forward we will have a 15-30 minute introduction call to discuss mutual expectations. An interview with the hiring manager will be next followed by an interview with key stakeholders.

We stand for a workplace that nurtures growth, fosters connection, and embraces authenticity. We believe that fostering a community that values diversity isn't just a goal; it's a way of life. The true magic happens when a group of unique minds comes together – bring yours and #EnergizeYourCareer"""

In [17]:
url = 'https://dev.api.talentmarx.in/api/v1/ml/similarity/'

In [18]:
def create_info_text(df):
    strings = []
    for _, row in df.iterrows():
        s = ""
        keys = row.keys()
        for i, element in enumerate(row):
            s += keys[i] + ": " + str(element).strip() + "\n"
        strings.append(s)
    return strings

In [19]:
text = create_info_text(df)

In [20]:
text[0]

"gender: nan\ndegree: bachelor of technology\ninstitute: uptu\nyear: 2002\ncity: bareilly\nemployer: avida technologies\nexperience: 0.0\nexperience_range: GREATER THAN 10 YEAR\nkeywords: ['Sensitive', 'Functional Requirements Specification', 'Database Design and Implementation', 'SQL Server Integration Services', 'Software Development Life Cycle', 'Data Transformation Services', 'Microsoft Enterprise Library', 'Requirement Specifications', 'Design and Implementation', 'Designing Architecture', 'Gathering Requirements', 'Object Oriented Design', 'Requirements Gathering', 'Business Requirements', 'MS Enterprise Library', 'Project Deliverables', 'Software Development', 'Improve Performance', 'SQL Server Database', 'Material Selection', 'Crystal Reports 9', 'Data Access Layer', 'High-Level Design', 'Operating Systems', 'Stored Procedures', 'Database Caching', 'Entity Framework', 'Full Text Search', 'Software Upgrade', 'Coding Standard', 'Design Patterns', 'Infragistics UI', 'SQL Server 20

In [21]:
df['gender'].unique().tolist()

[nan, 'male', 'female']

In [22]:
df['gender'].value_counts()

gender
male      29
female     4
Name: count, dtype: int64

In [23]:
df['city'].value_counts()

city
bangalore         11
chennai            6
hyderabad          5
mumbai             4
greater noida      4
bengaluru          4
coimbatore         3
pune               3
kannur             2
ghaziabad          2
noida              2
phagwara           1
hyd                1
nagpur             1
bareilly           1
theni              1
delhi              1
sultanpur          1
tirunelveli dt     1
karimnagar         1
karur              1
madurai            1
kottayam           1
new delhi          1
jaipur             1
ahmedabad          1
jamshedpur         1
patna              1
gurugram           1
anantapur          1
kakinada           1
trichy             1
erode              1
dharuhera          1
udumalpet          1
tirupur            1
Name: count, dtype: int64

In [24]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            33 non-null     object 
 1   degree            97 non-null     object 
 2   institute         92 non-null     object 
 3   year              70 non-null     object 
 4   city              71 non-null     object 
 5   employer          101 non-null    object 
 6   experience        108 non-null    float64
 7   experience_range  108 non-null    object 
 8   keywords          108 non-null    object 
 9   age               70 non-null     float64
dtypes: float64(2), object(8)
memory usage: 8.6+ KB


In [25]:
df['gender'].value_counts()

gender
male      29
female     4
Name: count, dtype: int64

In [26]:
df['gender'].unique()[0] == np.nan

False

In [27]:
def scale_score(x):
    return (x+1) / 2

In [28]:
def find_score(colname):
  for val in df[colname].unique().tolist():
    if 'nan' in str(val):
      continue
    temp_df = df.copy()
    temp_df[colname] = val
    resume_data = create_info_text(temp_df[['age', 'city', 'gender', 'institute', 'degree',
       'employer', 'experience']])
    data = {
      "queryDocumentString": jd,
      "documentStrings": resume_data
    }
    response = requests.post(url, json=data)
    scores = eval(response.text)['similarities']
    scores = list(map(scale_score, scores))
    df["Score_{}_{}".format(colname, val)] = scores
  return "Added Columns"

In [29]:
find_score('gender')

'Added Columns'

In [30]:
find_score('city')

'Added Columns'

In [31]:
find_score('institute')

C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

'Added Columns'

In [32]:
find_score('employer')

C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

'Added Columns'

In [33]:
find_score('degree')

C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df["Score_{}_{}".format(colname, val)] = scores
C:\Users\shour\AppData\Local\Temp\ipykernel_26496\2781604765.py:16: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performan

'Added Columns'

In [34]:
df.head()

,gender,degree,institute,year,city,employer,experience,experience_range,keywords,age,...,Score_degree_bachelor of computer science,Score_degree_master of computer application,Score_degree_higher secondary school certificate,Score_degree_bachelor of arts,Score_degree_bachelor of computer applications,Score_degree_master of science,Score_degree_master of commerce,Score_degree_doctor of medicine,Score_degree_master of technology,Score_degree_master of computer
0,NaN,bachelor of technology,uptu,2002,bareilly,avida technologies,0.0,GREATER THAN 10 YEAR,"[Sensitive, Functional Requirements Specificat...",0.0,...,0.603195,0.594427,0.596178,0.594128,0.597515,0.592949,0.592736,0.594020,0.593463,0.594084
1,male,bachelor of engineering,college of engineering and technology,1993,chennai,mercantiler infotech pvt ltd,0.0,8-10 YEAR,"[Microsoft Visual Studio, User Experience Desi...",0.0,...,0.579157,0.582245,0.581182,0.578568,0.585827,0.568573,0.570859,0.583250,0.573819,0.579262
2,male,bachelor of engineering,bharathiyar university,1982,bangalore,infinite computer solution,0.0,GREATER THAN 10 YEAR,"[Human Resources Management System, Hospital M...",1.0,...,0.611298,0.622934,0.621046,0.606161,0.630686,0.608349,0.611754,0.584972,0.616847,0.617145
3,NaN,bachelor of technology,jawaharlal nehru technological university,NaN,NaN,for five,0.0,3-5 YEAR,"[Confident, Manhattan Warehouse Management Sys...",NaN,...,0.553542,0.573648,0.577383,0.563328,0.569598,0.567224,0.570334,0.561907,0.566622,0.565068
4,NaN,master of computer applications,telangana university,NaN,NaN,cybage software pvt ltd,0.0,GREATER THAN 10 YEAR,"[Hard Working, Independent, Mirroring, Potenti...",NaN,...,0.568678,0.582926,0.578173,0.574453,0.577421,0.581984,0.581716,0.570633,0.580841,0.580052


In [35]:
df.describe()

,experience,age,Score_gender_male,Score_gender_female,Score_city_bareilly,Score_city_chennai,Score_city_bangalore,Score_city_coimbatore,Score_city_udumalpet,Score_city_dharuhera,...,Score_degree_bachelor of computer science,Score_degree_master of computer application,Score_degree_higher secondary school certificate,Score_degree_bachelor of arts,Score_degree_bachelor of computer applications,Score_degree_master of science,Score_degree_master of commerce,Score_degree_doctor of medicine,Score_degree_master of technology,Score_degree_master of computer
count,108.000000,70.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,...,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000,108.000000
mean,0.009259,0.514286,0.583698,0.587968,0.592519,0.587349,0.580995,0.590751,0.591687,0.575609,...,0.577849,0.587644,0.584326,0.580205,0.587601,0.577809,0.581337,0.569136,0.580399,0.583188
std,0.096225,0.503405,0.019549,0.020953,0.018851,0.019446,0.019421,0.019272,0.021393,0.020292,...,0.019816,0.020474,0.020778,0.017931,0.021623,0.019722,0.019329,0.016593,0.019938,0.019766
min,0.000000,0.000000,0.520613,0.515094,0.520623,0.499154,0.503173,0.524372,0.523406,0.512203,...,0.517975,0.510144,0.508879,0.523415,0.514914,0.509079,0.501875,0.526168,0.505627,0.508201
25%,0.000000,0.000000,0.571447,0.575935,0.585301,0.579618,0.572602,0.582079,0.581511,0.564515,...,0.564653,0.579033,0.574701,0.570627,0.575778,0.567155,0.570846,0.556858,0.571721,0.573692
50%,0.000000,1.000000,0.587035,0.590124,0.593081,0.587952,0.582039,0.593595,0.595366,0.579775,...,0.578956,0.588732,0.583461,0.579902,0.588998,0.578762,0.582829,0.568318,0.581472,0.585612
75%,0.000000,1.000000,0.597278,0.602809,0.604033,0.598736,0.593840,0.603210,0.604513,0.587511,...,0.592380,0.598777,0.595479,0.589903,0.600918,0.590429,0.592854,0.581251,0.591296,0.595413
max,1.000000,1.000000,0.630105,0.636965,0.635428,0.621289,0.617494,0.627908,0.632323,0.612934,...,0.615818,0.630105,0.631813,0.619950,0.631527,0.617729,0.618721,0.604845,0.628441,0.627417


In [36]:
df.columns

Index(['gender', 'degree', 'institute', 'year', 'city', 'employer',
       'experience', 'experience_range', 'keywords', 'age',
       ...
       'Score_degree_bachelor of computer science',
       'Score_degree_master of computer application',
       'Score_degree_higher secondary school certificate',
       'Score_degree_bachelor of arts',
       'Score_degree_bachelor of computer applications',
       'Score_degree_master of science', 'Score_degree_master of commerce',
       'Score_degree_doctor of medicine', 'Score_degree_master of technology',
       'Score_degree_master of computer'],
      dtype='object', length=214)

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 108 entries, 0 to 107
Columns: 214 entries, gender to Score_degree_master of computer
dtypes: float64(206), object(8)
memory usage: 180.7+ KB


In [38]:
from reportlab.lib.pagesizes import letter
from reportlab.lib import colors
from reportlab.lib.styles import getSampleStyleSheet
from reportlab.platypus import SimpleDocTemplate, Paragraph, Spacer, Image, PageBreak

doc = SimpleDocTemplate("report.pdf", pagesize=letter)
styles = getSampleStyleSheet()
title_style = styles['Title']
heading_style = styles['Heading1']
body_style = styles['BodyText']
elements = []

for col in ['gender','institute', 'city', 'employer', 'degree']:
    title =  Paragraph("Bias Analysis for {}".format(col), title_style)
    elements.append(title)
    elements.append(Spacer(1, 12))

    col_list = []
    for val in df[col].unique().tolist():
        if 'nan' not in str(val):
            col_list.append('Score_'+col+'_'+str(val))
    means = []
    for colname in col_list:
        mean = df[colname].mean()
        means.append(mean)
    en_means = list(enumerate(means))
    en_means.sort(key=lambda x: x[1], reverse=True)
    min_idx = []
    max_idx = []
    if len(en_means) > 3:
        for i in range(3):
            min_idx.append(en_means[-(i + 1)][0])
            max_idx.append(en_means[i][0])
        min_elements = [(col_list[i], means[i]) for i in min_idx]
        max_elements = [(col_list[i], means[i]) for i in max_idx]
    
        sec1 = Paragraph("Top 3 Categories with Minimum Mean Similarity Score: ", heading_style)
        elements.append(sec1)
        elements.append(Spacer(1, 12))
        for elem in min_elements:
            elements.append(Paragraph(f"{elem[0]} , {elem[1]}", body_style))
            elements.append(Spacer(1, 12))
        elements.append(Spacer(1, 12))
    
        sec2 = Paragraph("Top 3 Categories with Maximum Mean Similarity Score: ", heading_style)
        elements.append(sec2)
        elements.append(Spacer(1, 12))
        for elem in max_elements:
            elements.append(Paragraph(f"{elem[0]} , {elem[1]}", body_style))
            elements.append(Spacer(1, 12))
        elements.append(Spacer(1, 12))

    else:
        sec1 = Paragraph("All Categories with Mean Similarity Score: ", heading_style)
        elements.append(sec1)
        elements.append(Spacer(1, 12))
        for elem in [(col_list[i], means[i]) for i in range(len(means))]:
            elements.append(Paragraph(f"{elem[0]} , {elem[1]}", body_style))
            elements.append(Spacer(1, 12))
        elements.append(Spacer(1, 12))

    data = [df[colname] for colname in col_list]
    p_value = f_oneway(*data)[1]
    print(p_value)

    p_val = Paragraph(f"P-Value: {p_value}", body_style)
    elements.append(p_val)
    elements.append(Spacer(1, 12))

    if p_value < 0.05:
        plt.figure(figsize=(10, 12))
        plt.plot(col_list, means,)
        plt.xticks(rotation=90, fontsize = 6)
        plt.savefig('plot.png')
        plt.close()
        
        elements.append(Paragraph("Graphical Representation of all Scores", heading_style))
        elements.append(Spacer(1, 12))
        elements.append(Image('plot.png', width=500, height=500))
        elements.append(Spacer(1, 12))

        heading_style.textColor = colors.red
        elements.append(Paragraph("Bias Detected", heading_style))
        heading_style.textColor = colors.black
        elements.append(Spacer(1, 12))

    else:
        heading_style.textColor = colors.green
        elements.append(Paragraph("No Bias Detected", heading_style))
        heading_style.textColor = colors.black
        elements.append(Spacer(1, 12))

    elements.append(Spacer(1, 12))
    elements.append(PageBreak())

doc.build(elements)

0.12300857337954499
0.0
3.8755704730007516e-87
0.0
1.5593749638229604e-17
